## Computation Tools for Data Science - Group 6

**Group members:**
- Raquel Chaves Martinez - s243297
- Paula Gambus i Moreno - s233219
- Angel Paisan Garcia - s232793
- Alba Pi Mas - s243280

### Library importation

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import functions as f

import string
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


## Building and exploring the dataset

**The result of the merge of the datasets and calculation of the sentiment can be found in *main_complete.csv*. We recommend not rerunning the code used to generating the file because the computations involved are highly resource-intensive, resulting in long execution times.**

First, we open all the datasets that we are going to get information from.

In [2]:
netflix = pd.read_csv('data/netflix.csv')
amazon = pd.read_csv('data/amazon.csv')
hulu = pd.read_csv('data/hulu.csv')
apple = pd.read_csv('data/apple.csv')
hbo = pd.read_csv('data/hbo.csv')

main = pd.read_csv('data/TMDB_all_movies.csv')

Since we only want to do movie recommendation, we remove all the entries that are not classified as movies in the platform datasets.

In [3]:
print('Before removing shows')
print(netflix.shape)
print(amazon.shape)
print(hulu.shape)
print(apple.shape)
print(hbo.shape)

netflix = f.remove_tv_show(netflix)
apple = f.remove_tv_show(apple)
hulu = f.remove_tv_show(hulu)
amazon = f.remove_tv_show(amazon)
hbo = f.remove_tv_show(hbo)

#dimension of the dataset
print('After removing shows')
print(netflix.shape)
print(amazon.shape)
print(hulu.shape)
print(apple.shape)
print(hbo.shape)

Before removing shows
(20106, 8)
(66905, 8)
(9753, 8)
(17648, 8)
(7017, 8)
After removing shows
(15499, 8)
(58559, 8)
(5915, 8)
(13598, 8)
(3653, 8)


Once we have the auxiliar datasets clean, we can merge it with the main dataset. To do it, we have built a function that can be found in *function.py* . This function takes the platform dataset and the main dataset and adds a new binary column where the 0 indicates that the movie is not in the platform and the 1 represents that the movie is in the platform.

In [4]:
main_netflix = f.platform_column(main, netflix, 'Netflix')
main_net_ama = f.platform_column(main_netflix, amazon, 'Amazon')
main_net_ama_hulu = f.platform_column(main_net_ama, hulu, 'Hulu')
main_net_ama_hulu_apple = f.platform_column(main_net_ama_hulu, apple, 'Apple')
main_complete = f.platform_column(main_net_ama_hulu_apple, hbo, 'HBO')


To see the amount of movies that we have in each platform:

In [5]:
netflix_count = f.count_movies(main_complete, 'Netflix')
print('Movies in Netflix:', netflix_count)
amazon_count = f.count_movies(main_complete, 'Amazon')
print('Movies in Amazon prime:', amazon_count)
hulu_count = f.count_movies(main_complete, 'Hulu')
print('Movies in Hulu:', hulu_count)
apple_count = f.count_movies(main_complete, 'Apple')
print('Movies in Apple TV:', apple_count)
hbo_count = f.count_movies(main_complete, 'HBO')
print('Movies in HBO Max:', hbo_count)

Movies in Netflix: 36998.0
Movies in Amazon prime: 108669.0
Movies in Hulu: 15330.0
Movies in Apple TV: 32572.0
Movies in HBO Max: 10606.0


Once we have all the information in the same dataframe, we can start applying methods. First, we will calculate the sentiment of the overview of the movies.

## Calculation of the sentiment

First, we will do some exploration of the overview.

In [7]:
main_complete['overview']

0          After the coal mine he works at closes and his...
1          Nikander, a rubbish collector and would-be ent...
2          It's Ted the Bellhop's first night on the job....
3          Four young friends, while taking a shortcut en...
4          Timo Novotny labels his new project an experim...
                                 ...                        
1009834    “Hommage à Beksinski” is a short film commissi...
1009835                                                  NaN
1009836                                                  NaN
1009837                                                  NaN
1009838                                                  NaN
Name: overview, Length: 1009839, dtype: object

As it can be seen, there are some missing values in the overview (description of the movie). When we calculate the sentiment, the movies that don't have an overview will be assigned Nan and not 0, because a sentiment of 0 denotes sadness (the lower the score, the sadder is the description/movie).

We have built a sentiment function that takes a list of tokens and calculates the sentiment based on the LabMT wordlist. This function is called by a fucntion named *preprocess_and_analyze_sentiment*, which takes the text and returns the sentiment score.

In [8]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
punctuation_table = str.maketrans('', '', string.punctuation)

In [9]:
main_complete['sentiment'] = main_complete['overview'].apply(f.preprocess_and_analyze_sentiment)

In [ ]:
#save the main_complete dataset
#main_complete.to_csv('data/main_complete.csv', index=False)

## Exploration of the dataset to calculate similarities

After merging the movies dataset with the platforms dataset and adding the sentiment score columnd we saved the resulting dataset ('main_complete') to avoid generating it every time. Here, we load the dataset and continue working.